## Oxford5k dataset:

In [62]:
train_bovw, test_bovw, train_colour_images, train_names = load_oxford_data()

btn_upload = widgets.FileUpload()
btn_show = widgets.Button(description='Display Image:')
btn_run = widgets.Button(description='Retrieve Images:')
btn_show.on_click(find_image)
btn_run.on_click(retrieve_image)
out_pl = widgets.Output()
out_pl.clear_output()

VBox([widgets.Label('Image Retrieval Oxford5k:'), btn_upload, btn_show, btn_run, out_pl])

## MPEG7 dataset:

In [63]:
mpeg_images, filenames, pixel_values = load_mpeg_data()

btn_upload_mpeg = widgets.FileUpload()
btn_show_mpeg = widgets.Button(description='Display Image:')
btn_run_mpeg = widgets.Button(description='Retrieve Images:')
btn_show_mpeg.on_click(find_mpeg_image)
btn_run_mpeg.on_click(retrieve_mpeg_image)
out_pl_mpeg = widgets.Output()
out_pl_mpeg.clear_output()

VBox([widgets.Label('Image Retrieval MPEG7:'), btn_upload_mpeg, btn_show_mpeg, btn_run_mpeg, out_pl_mpeg])

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

## 

In [53]:
import numpy as np 
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics.pairwise import euclidean_distances
from fastai.vision.widgets import *
from fastai.vision.all import *

In [54]:
def load_test_data(images_path, gt_path):
    test_images_path = []
    test_names = []
    test_gray_images = []
    test_colour_images = []
    
    for filename in sorted(os.listdir(gt_path)):
        if filename.endswith("query.txt"):

            # Saving filename
            tmp = filename.split(".")[0].split("_")
            if len(tmp) == 4:
                name = tmp[0]+"_"+tmp[1]
            elif len(tmp) == 3:
                name = tmp[0]
            test_names.append(name)

            # Reading the image number to be saved
            with open(os.path.join(gt_path, filename), "r") as f:
                line = f.readline()
                test_images_path.append(line.split(" ")[0])

    for path in test_images_path:
        image = cv2.imread(os.path.join(images_path, path[5:]) + ".jpg")
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        test_gray_images.append(gray_image)
        test_colour_images.append(image)
    
    #print("Loaded in {} Images!".format(len(test_gray_images)))
    
    return test_gray_images, test_colour_images, test_names

In [55]:
def load_train_data(images_path, gt_path):
    train_names = []
    train_images_path = []
    train_gray_images = []
    train_colour_images = []

    all_image_names = []

    for filename in sorted(os.listdir(gt_path)):
        if filename.endswith("good.txt") or filename.endswith("ok.txt"):

            # Saving filenames
            tmp = filename.split(".")[0].split("_")
            if len(tmp) == 4:
                name = tmp[0]+"_"+tmp[1]
            elif len(tmp) == 3:
                name = tmp[0]

            # Saving image paths
            with open(os.path.join(gt_path, filename), "r") as f:
                line = f.readlines()
                for i in range(len(line)):
                    line[i] = line[i][:-1]
                    # Check if image has already been added:
                    if line[i] not in all_image_names:
                        # Append this many names
                        train_names.append(name)
                        train_images_path.append(line[i])
                        all_image_names.append(line[i])

    for path in train_images_path:
        image = cv2.imread(os.path.join(images_path, path) + ".jpg")
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        train_gray_images.append(gray_image)
        train_colour_images.append(image)
    
    #print("Loaded in {} Images!".format(len(train_gray_images)))
    
    return train_gray_images, train_colour_images, train_names

In [56]:
def load_MPEG7_data(folder, image_type, gray=False):
    images = []
    filenames = []
    y = []
    category, idx = "none", 0
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(image_type):
            if image_type == ".gif":
                gif_image = cv2.VideoCapture(os.path.join(folder, filename))
                ret, frame = gif_image.read()
                image = Image.fromarray(frame)
                image = image.resize((64, 64), Image.ANTIALIAS)
                image = np.array(image)
            else:
                image = cv2.imread(os.path.join(folder, filename))
            if gray:
                gray_image = image
            else:
                gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if gray_image is not None:
                images.append(gray_image)
                split_name = filename.split('_')
                if len(split_name) == 3:
                    fname = filename.split('_')[0] + "_" + filename.split('_')[1]
                elif len(split_name) == 2:
                    fname = filename.split('_')[0]
                elif len(split_name) == 1:
                    fname = filename.split('-')[0]
                filenames.append(fname)
                if filename.startswith(category):
                    y.append(idx)
                else:
                    split = filename.split('_')
                    if len(split) == 3:
                        category = filename.split('_')[0] + "_" + filename.split('_')[1]
                    elif len(split) == 2:
                        category = filename.split('_')[0]
                    idx = idx + 1
                    y.append(idx)
    #print(len(images), "Images loaded successfully!")
    return images, filenames

In [57]:
def find_pixel_values(test_gray_images, train_gray_images):
    test_pixels = []
    for image in test_gray_images:
        img = Image.fromarray(image)
        img = img.resize((100,100))
        img = np.array(img)
        img = img.reshape((10000,))
        test_pixels.append(img)

    train_pixels = []
    for image in train_gray_images:
        img = Image.fromarray(image)
        img = img.resize((100,100))
        img = np.array(img)
        img = img.reshape((10000,))
        train_pixels.append(img)
    
    return test_pixels, train_pixels

In [58]:
def mpeg_find_pixel_values(images):
    tmp = []
    for image in images:
        im = np.asarray(image)
        im = im.reshape((4096,))
        tmp.append(im)
    pixel_values = np.array(tmp)
    return pixel_values

In [59]:
def image_retrieval_k(train_data, test_data, train_names, test_names, train_images_as_array, test_images_as_array, k=20, view_option=0, image_size=(32,32)):
    avg_precisions = []
    avg_recalls = []
    precisionsatk = []
    count = 0
    
    for idx, query in enumerate(test_data):
        
        all_precisions = []
        all_recalls = []
        precisions = []
        recalls = []

        # Finding the euclidean distance from the query image and sorting them into index
        query = query.reshape((1, -1))
        D = euclidean_distances(train_data, query).squeeze()
        index = np.argsort(D)
        
        # Finding the index of the last correct image in the sorted index to iter to
        last_correct_image_idx = 0
        for i in range(len(index)):
            if train_names[index[i]] == test_names[idx]:
                last_correct_image_idx = i
        
        # make sure we iter to k (for precision@k) if all correct images are found before k
        if k > last_correct_image_idx:
            last_correct_image_idx = k+1
        
        # Itering through all images untill we get to k or last correct image to compute AP
        for kk in range(1, last_correct_image_idx+2):
            TP = 0
            FP = 0
            FN = 0
            
            # Finding the correct amount of images in the training set
            correct_count = 0
            for ind in index:
                if train_names[ind] == test_names[idx]:
                    correct_count += 1
            sized_index = index[:kk]
            
            # Find TP FP FN
            for ind in sized_index:
                if train_names[ind] == test_names[idx]:
                    TP += 1
                else:
                    FP += 1
            FN = correct_count - TP
            
            # If we want to view the images then we run this code, else its a waste of computational time
            if view_option == 1:
                # Creating image of k images (including query image at start)
                tmp = [query.reshape(image_size)]
                for ind in sized_index[:k]:
                    tmp.append(train_data[ind].reshape(image_size))
                output = np.array(tmp)*255
                output = output.transpose(1, 0, 2)
                output = output.reshape((image_size[0], -1))
                im_query = Image.fromarray(output)
            
            # If the last k image is a correct image we add precision to the list
            if train_names[sized_index[-1]] == test_names[idx]:
                precisions.append(TP/(TP+FP))
                recalls.append(TP/(TP+FN))

            # Adding all precisions and recalls to a seperate list
            all_precisions.append(TP/(TP+FP))
            if TP+FN != 0:
                all_recalls.append(TP/(TP+FN))
        
     
        # Solving AP, AR and precision@k
        avg_precisions.append(np.average(precisions))
        avg_recalls.append(np.average(all_recalls))
        precisionsatk.append(all_precisions[k-1])
        
        # Set a viewing option, if 1 we print out the following:
        if view_option == 1:
            display(im_query) 
            print("Label: {}".format(test_names[idx]))
            print("Average Precision for query {}: ".format(idx), avg_precisions[-1])
            print("Precision@k for query {}: ".format(idx), precisionsatk[-1])
            print("\n")
        elif view_option == 0:
            count += 1 
            print("Percentage Complete: {}".format(round((count/len(test_data))*100),2), end="\r")
        elif view_option == 2:
            # Allowing a view_option 2 -> for viewing top k images from non_pixel value inputs
            # creating an array of the top k similar images
            top_k_images = [test_images_as_array[idx]]
            for i in range(0,k):
                top_k_images.append(train_images_as_array[index[i]])

            fig, axes = plt.subplots(1, k+1, figsize=(250/k, 250/k))
            for i, (image, ax) in enumerate(zip(top_k_images, axes.ravel())):
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)####
                ax.imshow(image, cmap="gray")
                if i == 0:
                    ax.set_title("Query: {}".format(test_names[idx]))
                else:
                    ax.set_title(train_names[sized_index[i-1]])
                ax.axis("off")
            plt.show()
            #print("Label: {}".format(test_names[idx]))
            print("Average Precision for query {}: ".format(idx), avg_precisions[-1])
            print("Precision@k for query {}: ".format(idx), precisionsatk[-1])
            print("\n")
        
    return avg_precisions, avg_recalls, precisionsatk, top_k_images

In [60]:
def load_oxford_data():
    # Load in data x No. of words:
    train_bovw = np.load("SIFT/NPY files for BoVW/bovw files for 100000 Words/BoW_Train.npy")
    test_bovw = np.load("SIFT/NPY files for BoVW/bovw files for 100000 Words/BoW_Test.npy")

    # Load in all images (training, testing)
    images_path = r"C:\Users\Sean\Desktop\Image-Retrival (clean)\SIFT\Oxford building images"
    gt_path = r"C:\Users\Sean\Desktop\Image-Retrival (clean)\SIFT\Ground Truth files"
    train_gray_images, train_colour_images, train_names = load_train_data(images_path, gt_path)
    
    return train_bovw, test_bovw, train_colour_images, train_names

def find_image(change):
    name = btn_upload.metadata[-1]['name'].split("-")[0].split(".")[0]
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: 
        display(img.to_thumb(128,128))
        print(name)

def retrieve_image(change):
    name = btn_upload.metadata[-1]['name'].split("-")[0].split(".")[0]
    idx = btn_upload.metadata[-1]['name'].split("-")[-1].split(".")[0]
    img = PILImage.create(btn_upload.data[-1])
    img = np.asarray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out_pl.clear_output()
    with out_pl:
        AP, AR, precisionsatk, top_k_images = image_retrieval_k(train_bovw, test_bovw[int(idx):int(idx)+1], train_names, [name], train_colour_images, [img], 10, 2)

In [61]:
def load_mpeg_data():
    images_folder = r"C:\Users\Sean\Desktop\Image-Retrival (clean)\MPEG7\MPEG7"
    mpeg_images, filenames = load_MPEG7_data(images_folder, ".gif", False)
    pixel_values = mpeg_find_pixel_values(mpeg_images)
    
    return mpeg_images, filenames, pixel_values

def find_mpeg_image(change):
    name = btn_upload_mpeg.metadata[-1]['name'].split("-")[0]
    img = PILImage.create(btn_upload_mpeg.data[-1])
    out_pl_mpeg.clear_output()
    with out_pl_mpeg: 
        display(img.to_thumb(128,128))
        print(name)

def retrieve_mpeg_image(change):
    test_name = btn_upload_mpeg.metadata[-1]['name'].split("-")[0]
    img = PILImage.create(btn_upload_mpeg.data[-1])
    test_image = np.asarray(img)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    test_image = cv2.resize(test_image, (64,64))
    test_pixel = test_image.reshape((4096,))
    out_pl_mpeg.clear_output()
    with out_pl_mpeg:
        AP, AR, precisionsatk, top_k_images = image_retrieval_k(pixel_values, [test_pixel], filenames, [test_name], mpeg_images, [test_image], 10, 2)